In [ ]:
import torch 
from torch.utils.data import DataLoader
from src import full_pcd_dataset
import importlib
importlib.reload(full_pcd_dataset)
import json
# Create dataset instance
dataset = full_pcd_dataset.FullPCDDataset("data/full_pcd_30000_samples_6d.npz")
with open("label_dict.json", "r") as f:
    label_dict = json.load(f)
num_classes = len(label_dict.keys())
# Split into train and validation (example: 80-20 split)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

# Create data loaders
batch_size = 128
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=8,pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=8,pin_memory=True)
num_classes

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


5

In [2]:
train_dataset[0][1][0]

tensor(1)

In [3]:
from src.model import PoseWithClassModel
from src.model_classification import ClassificationModel 
from torch.optim.lr_scheduler import StepLR
model = ClassificationModel(num_classes)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
scheduler = StepLR(optimizer, step_size=10, gamma=0.5) 


In [4]:
# from src.train_utils import train_model
from src.train_classificator import train_model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


Using device: cuda


In [5]:
train_model(model, train_loader, val_loader, optimizer, scheduler, device, epochs=30, directory="new_run_nbn")

Training: 100%|██████████| 188/188 [00:43<00:00,  4.33it/s]


Epoch 1/30 - Total Loss: 0.2944


Validating: 100%|██████████| 47/47 [00:22<00:00,  2.08it/s]


Validation - Loss: 128.2609 | Accuracy: 20.12%


Training: 100%|██████████| 188/188 [00:43<00:00,  4.35it/s]


...

Epoch 5/30 - Total Loss: 0.0218


Validating: 100%|██████████| 47/47 [00:22<00:00,  2.05it/s]


Validation - Loss: 95.3936 | Accuracy: 20.12%


Training:   0%|          | 0/188 [00:08<?, ?it/s]


KeyboardInterrupt: 

In [6]:
model

ClassificationModel(
  (backbone): PointNetPPBackbone(
    (sa1): PointNetSetAbstraction(
      (mlp): ModuleList(
        (0): Conv2d(3, 64, kernel_size=(1, 1), stride=(1, 1))
        (1): ReLU(inplace=True)
        (2): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
        (3): ReLU(inplace=True)
        (4): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1))
        (5): ReLU(inplace=True)
      )
    )
    (sa2): PointNetSetAbstraction(
      (mlp): ModuleList(
        (0): Conv2d(131, 128, kernel_size=(1, 1), stride=(1, 1))
        (1): ReLU(inplace=True)
        (2): Conv2d(128, 128, kernel_size=(1, 1), stride=(1, 1))
        (3): ReLU(inplace=True)
        (4): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1))
        (5): ReLU(inplace=True)
      )
    )
    (sa3): PointNetSetAbstraction(
      (mlp): ModuleList(
        (0): Conv2d(259, 256, kernel_size=(1, 1), stride=(1, 1))
        (1): ReLU(inplace=True)
        (2): Conv2d(256, 512, kernel_size=(1, 1), stride=(1, 1))

In [ ]:
# one_pcd = dataset[2][0]

# model.eval()
# with torch.no_grad():
#     # Move the point cloud to the same device as the model
#     one_pcd = one_pcd.to(device)
#     # Forward pass through the model
#     # Note: Add batch dimension
#     class_logits, pred_pose = model(one_pcd.unsqueeze(0).to(device))
# one_pcd = o3d.geometry.PointCloud(o3d.utility.Vector3dVector(one_pcd.cpu().numpy()))
# one_pcd.paint_uniform_color([0.5, 0.5, 0.5])

In [ ]:
from pathlib import Path
import open3d as o3d
import numpy as np
from src.train_utils import rot6d_to_matrix, geodesic_loss_numpy, rot6d_to_matrix_numpy
model_dir = Path("./data/models")
model_list = list(model_dir.glob("*"))

def check_model_on_instance(model, dataset, idx, device, paths):
    pcd = dataset[idx][0]
    gt_class = dataset[idx][1][0]
    gt_rotation_6d = dataset[idx][1][1]
    gt_rotation = rot6d_to_matrix(gt_rotation_6d)
    gt_translation = dataset[idx][1][2]
    gt_pcd = o3d.geometry.PointCloud(o3d.utility.Vector3dVector(pcd))
    
    model.eval()
    with torch.no_grad():
        # Move the point cloud to the same device as the model
        pcd = pcd.to(device)
        # Forward pass through the model
        # Note: Add batch dimension
        class_logits, pred_pose = model(pcd.unsqueeze(0).to(device))
        preds = class_logits.argmax(dim=1)
        model_path = paths[preds.item()]
        print(f"Predicted class: {preds.item()}", 
              f"GT class: {gt_class.item()}")
        print(f"Model path: {model_path}")
    pcd_predicted = o3d.io.read_triangle_mesh(model_path)
    pcd_predicted.paint_uniform_color([1, 0, 0])  # Red color for predicted model
    predicted_rotation_6d = pred_pose[:, 3:]
    rotation = rot6d_to_matrix(predicted_rotation_6d)[0].cpu().numpy()

    translation = pred_pose[0, :3].cpu().numpy()
    transformation = np.vstack((np.array([0,0,0,1]),np.hstack((rotation, translation.reshape(3, 1)))))

    # predicted_quaternion = pred_pose[0, 3:].cpu().numpy()

    # q1, q2, q3, q4 = predicted_quaternion/np.linalg.norm(predicted_quaternion)
    # print(np.linalg.norm(np.array([q1, q2, q3, q4])))
    # x, y, z = pred_pose[0, :3].cpu().numpy()
    # # Convert quaternion to rotation matrix
    # T = np.array([
    #     [1 - 2*(q3**2 + q4**2),     2*(q2*q3 - q1*q4),
    #         2*(q2*q4 + q1*q3), x],
    #     [2*(q2*q3 + q1*q4), 1 - 2*(q2**2 + q4**2),     2*(q3*q4 - q1*q2), y],
    #     [2*(q2*q4 - q1*q3),     2*(q3*q4 + q1*q2), 1 - 2*(q2**2 + q3**2), z],
    #     [0, 0, 0, 1]
    # ])
    pcd_predicted.transform(transformation)
    translation_diff = translation- gt_translation.numpy()
    print(f"Translation diff: {translation_diff}")
    translation_error = sum(translation - gt_translation.cpu().numpy())**2)/3
    print(f"Translation error: {translation_error}")
    # rotation_error = 1 - np.dot(predicted_quaternion, gt_rotation_quaternion)
    rotation_error = geodesic_loss_numpy(rotation, gt_rotation)
    print(f"Rotation error: {rotation_error}")
    # Visualize the point cloud and the predicted model
    o3d.visualization.draw_geometries([pcd_predicted, gt_pcd])


In [ ]:
import numpy as np
rot = np.eye(3)
translation = np.array([0, 0, 1])
np.hstack((rot, translation.reshape(-1, 1)))

In [ ]:
translation

In [ ]:
check_model_on_instance(model, train_dataset, 0, device, model_list)

In [ ]:
preds = class_logits.argmax(dim=1)
preds.item()

In [ ]:



model_path = model_list[preds.item()]

In [ ]:
q1, q2, q3, q4 = pred_pose[0, 3:].cpu().numpy()
print(q1, q2, q3, q4)

In [ ]:

pcd_predicted = o3d.io.read_triangle_mesh(model_path)
pcd_predicted.paint_uniform_color([1, 0, 0])  # Red color for predicted model
translation = pred_pose[0, :3].cpu().numpy()
rotation = pred_pose[0, 3:].cpu().numpy()
pcd_predicted.rotate(pcd_predicted.get_rotation_matrix_from_quaternion(rotation))
pcd_predicted.translate(translation)

pcd_predicted

In [ ]:
((dataset[2][1][2] - translation)**2).sum()/3

In [ ]:
import numpy as np
print(dataset[2][1][1], rotation/np.linalg.norm(rotation))
1-np.dot(dataset[2][1][1], rotation/np.linalg.norm(rotation))

In [ ]:
np.linalg.norm(dataset[1][1][1])

In [ ]:
o3d.visualization.draw_geometries([pcd_predicted, one_pcd])